<a href="https://colab.research.google.com/github/harrymkwn/TensorFlow/blob/master/Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
x = tf.Variable(3,name="x")
y = tf.Variable(4,name="y")

In [4]:
print(x.value())

Tensor("x/read:0", shape=(), dtype=int32)


In [0]:
f = x*x*y + y + 2 

In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
  init.run()
  print(x)
  result = f.eval()
  print(result)

<tf.Variable 'x:0' shape=() dtype=int32_ref>
42


#Linear Regression with Tensorflow

In [0]:
from sklearn.datasets import fetch_california_housing

In [8]:
housing = fetch_california_housing()

In [9]:
m,n = housing.data.shape
print(m,n)

20640 8


In [10]:
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]
print(housing_data_plus_bias)

[[   1.            8.3252       41.         ...    2.55555556
    37.88       -122.23      ]
 [   1.            8.3014       21.         ...    2.10984183
    37.86       -122.22      ]
 [   1.            7.2574       52.         ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.            1.7          17.         ...    2.3256351
    39.43       -121.22      ]
 [   1.            1.8672       18.         ...    2.12320917
    39.43       -121.32      ]
 [   1.            2.3886       16.         ...    2.61698113
    39.37       -121.24      ]]


In [0]:
X = tf.constant(housing_data_plus_bias,dtype=tf.float32,name="X")
Y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="Y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),Y)

In [0]:
saver = tf.train.Saver()
from google.colab import drive
drive.mount('/content/gdrive/')


In [0]:
with tf.Session() as sess:
  theta_value = theta.eval()
  print(theta_value)
  save_path = saver.save(sess, "/final_model.ckpt")

#stochestic Gradient Descent 

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
data  = fetch_20newsgroups()

In [0]:
def generate_dataset():
 x_batch = np.linspace(0, 2, 100)
 y_batch = 1.5 * x_batch + np.random.randn(*x_batch.shape) * 0.2 + 0.5
 return x_batch, y_batch

In [0]:
def linear_regression():
  x = tf.placeholder(tf.float32, shape=(None, ), name='x')
  y = tf.placeholder(tf.float32, shape=(None, ), name='y')

  with tf.variable_scope('lreg') as scope:
    w = tf.Variable(np.random.normal(), name='W')
    b = tf.Variable(np.random.normal(), name='b')
		
    y_pred = tf.add(tf.multiply(w, x), b)

    loss = tf.reduce_mean(tf.square(y_pred - y))

  return x, y, y_pred, loss

In [0]:
def run():
  x_batch, y_batch = generate_dataset()
  x, y, y_pred, loss = linear_regression()

  optimizer = tf.train.GradientDescentOptimizer(0.1)
  train_op = optimizer.minimize(loss)

  with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    feed_dict = {x: x_batch, y: y_batch}
		
    for i in range(300):
      session.run(train_op, feed_dict)
      print(i, "loss:", loss.eval(feed_dict))

    print('Predicting')
    y_pred_batch = session.run(y_pred, {x : x_batch})

  plt.scatter(x_batch, y_batch)
  plt.plot(x_batch, y_pred_batch, color='red')
  plt.xlim(0, 2)
  plt.ylim(0, 2)
  plt.savefig('plot.png')


In [0]:
run()

In [0]:

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))